In [1]:
import pandas as pd
import altair as alt
import numpy as np
import openpyxl

In [17]:
nhex_file = pd.ExcelFile('C:/Users/rowan/Desktop/CPSC/Research Project/Data/nhex-series-b-2024-data-tables-en(2).xlsx')
nhex_total = nhex_file.parse("Prov-Terr Gov't", header=4, skiprows=lambda x: x > 54)
nhex_percapita = nhex_file.parse("Prov-Terr Gov't", header=113, skiprows=lambda x: x > 163)
nhex_percapita.head()

,Year,N.L.,P.E.I.,N.S.,N.B.,Que.,Ont.,Man.,Sask.,Alta.,B.C.,Y.T.,N.W.T,Nun.,Canada (Average)
0,1975,357.923507,352.587408,322.884971,300.867937,399.858506,377.830764,367.520427,329.308710,384.207843,371.337761,281.700749,355.889286,—,376.320519
1,1976,389.231372,382.682388,362.386939,351.601682,464.684666,429.351781,435.491898,390.816402,434.149625,427.287157,393.108596,427.601300,—,431.980872
2,1977,412.980589,419.500926,398.290593,390.487509,508.181706,462.030902,479.361249,434.853237,451.206536,464.188631,444.817915,525.961690,—,467.929843
3,1978,455.019387,463.222363,438.041067,427.555060,568.566309,492.354174,504.310051,468.116817,504.614559,521.507176,492.421298,619.819282,—,512.012063
4,1979,521.433145,508.778533,486.339411,478.804550,632.248226,527.104579,549.257331,529.786608,600.003290,583.985183,538.340153,610.039770,—,565.939181
